<a href="https://colab.research.google.com/github/koenvanwijk/Complete-Block-Level-Visual-Debugger-for-Blockly/blob/master/Download_blocklybench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://console.cloud.google.com/logs/query;cursorTimestamp=2024-09-09T12:25:08.122563076Z;duration=PT1H?project=motar-242711

download as json, and save as content.json


In [7]:
# prompt: open the json file content.json

import json

with open('content.json', 'r') as f:
  data = json.load(f)
  print(data)


[{'protoPayload': {'@type': 'type.googleapis.com/google.appengine.logging.v1.RequestLog', 'appId': 'e~motar-242711', 'versionId': '20230601t205730', 'requestId': '66deea0100ff0340f4750a3e4a0001657e6d6f7461722d3234323731310001323032333036303174323035373330000100', 'ip': '2a01:111:f400:fe1b::100', 'startTime': '2024-09-09T12:28:49.239227Z', 'endTime': '2024-09-09T12:28:49.339916Z', 'latency': '0.100689s', 'method': 'HEAD', 'resource': '/?editor=ui-plugin-editor', 'httpVersion': 'HTTP/1.1', 'status': 200, 'responseSize': '10159', 'host': 'motar-242711.ew.r.appspot.com', 'instanceIndex': -1, 'finished': True, 'appEngineRelease': '1.9.71', 'traceId': '6ec18b8e7b970b90288c64466881045c', 'first': True, 'traceSampled': True, 'spanId': 'b58422dca4c6be7c'}, 'insertId': '66deea010005306192273fc2', 'httpRequest': {'status': 200}, 'resource': {'type': 'gae_app', 'labels': {'project_id': 'motar-242711', 'version_id': '20230601t205730', 'zone': 'eu6', 'module_id': 'default'}}, 'timestamp': '2024-09-0

In [43]:
# prompt: walk the tree of the data to search for all the jsonPayload

def find_json_payloads(data):
  """
  Walks the tree of the given data to search for all the jsonPayloads.

  Args:
    data: The data to search.

  Returns:
    A list of all the jsonPayloads found.
  """
  json_payloads = []
  if isinstance(data, dict):
    for key, value in data.items():
      if key == "jsonPayload":
        timestamp = data['timestamp']
        json_payloads.append((timestamp, value))
      else:
        json_payloads.extend(find_json_payloads(value))
  elif isinstance(data, list):
    for item in data:
      json_payloads.extend(find_json_payloads(item))
  return json_payloads

json_payloads = find_json_payloads(data)
print(json_payloads)


[('2024-09-09T12:47:44.908352643Z', {'message': "TypeError: Cannot read properties of undefined (reading 'pathObject')\n    at WorkspaceSearch.highlightCurrentSelection_ (webpack://blocklybench/./node_modules/@blockly/plugin-workspace-search/src/index.js_+_2_modules?:716:31)\n    at highlight_blocks_of_type (webpack://blocklybench/./src/concrete_workspace.js?:317:36)\n    at Concrete.select_block_type (webpack://blocklybench/./src/concrete_workspace.js?:330:3)\n    at Concrete.mySelection (webpack://blocklybench/./src/concrete_workspace.js?:291:14)\n    at WorkspaceSvg$$module$build$src$core$workspace_svg.fireChangeListener (webpack://blocklybench/./node_modules/blockly/blockly_compressed.js?:1068:333)\n    at fireNow$$module$build$src$core$events$utils (webpack://blocklybench/./node_modules/blockly/blockly_compressed.js?:67:457)", 'serviceContext': {'service': 'web'}, '@type': 'type.googleapis.com/google.devtools.clouderrorreporting.v1beta1.ReportedErrorEvent', 'context': {'httpReques

In [44]:
# prompt: convert the message string to json

import json

def get_json_from_message(message):
  """
  Extracts the JSON object from a message string.

  Args:
    message: The message string containing the JSON object.

  Returns:
    The extracted JSON object as a Python dictionary.
  """
  try:
    # Remove leading and trailing whitespace
    message = message.strip()
    # Find the start and end indices of the JSON object
    start_index = message.find('{')
    end_index = message.rfind('}') + 1
    # Extract the JSON string
    json_string = message[start_index:end_index]
    # Parse the JSON string into a Python dictionary
    json_object = json.loads(json_string)
    return json_object
  except json.JSONDecodeError:
    print("Error: Invalid JSON string.")
    return None




In [45]:
# prompt: I have "at saveBlocks (webpack://blocklybench/./src/block_factory_workspace.js?:116:20)" please get the 116 out

def extract_number(text):
  """
  Extracts the number from a string like "at saveBlocks (webpack://blocklybench/./src/block_factory_workspace.js?:116:20)".

  Args:
    text: The input string.

  Returns:
    The extracted number as an integer, or None if no number is found.
  """
  try:
    # Split the string by ":" and get the second part
    parts = text.split(":")
    if len(parts) >= 2:
      # Extract the number from the second part
      number = int(parts[2])
      return number
  except ValueError:
    pass
  return None

# Example usage
text = "at saveBlocks (webpack://blocklybench/./src/block_factory_workspace.js?:116:20)"
number = extract_number(text)
print(number)  # Output: 116


116


In [60]:
cd /content

/content


In [61]:
# prompt: now walk the array and get the message out

for (timestamp,payload) in json_payloads:
  if "message" in payload:

    # timestamp is like 2024-09-09T12:56:18.176775923Z
    # strip to seconds
    timestamp = timestamp[:16]


    # create a folder if it does not exist with the timestamp
    import os
    print(f"created {timestamp}")
    if not os.path.exists(timestamp):
      os.makedirs(timestamp)
      print("created {timestamp}")


    message = payload["message"]

    mergeable_json = get_json_from_message(message)

    line_nr = extract_number(message[message.rfind('}') + 1:])
    map = {116: "factory",
           117: "toolbox",
           118: "codegen",
           119: "concrete"}
    if line_nr:
      # map the line_nr to file name

      # save the mergeable_json in a file determined bye the map
      with open(f"{timestamp}/{map[line_nr]}.json", "w") as f:
        f.write(json.dumps(mergeable_json))





created 2024-09-09T12:47
created {timestamp}
Error: Invalid JSON string.
created 2024-09-09T12:47
Error: Invalid JSON string.
created 2024-09-09T12:47
Error: Invalid JSON string.
created 2024-09-09T12:47
created 2024-09-09T12:47
created 2024-09-09T12:47
created 2024-09-09T12:47
created 2024-09-09T12:53
created {timestamp}
created 2024-09-09T12:53
created 2024-09-09T12:53
created 2024-09-09T12:53
created 2024-09-09T12:55
created {timestamp}
Error: Invalid JSON string.
created 2024-09-09T12:55
Error: Invalid JSON string.
created 2024-09-09T12:55
created 2024-09-09T12:55
created 2024-09-09T12:55
created 2024-09-09T12:55
created 2024-09-09T12:55
Error: Invalid JSON string.
created 2024-09-09T12:55
created 2024-09-09T12:55
created 2024-09-09T12:55
created 2024-09-09T12:55
created 2024-09-09T12:56
created {timestamp}
created 2024-09-09T12:56
created 2024-09-09T12:56
created 2024-09-09T12:56
created 2024-09-09T12:56
created 2024-09-09T12:56
created 2024-09-09T12:56
created 2024-09-09T12:56
cr